In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
df = pd.read_pickle("data/urfunny.pkl")
df

,sentence,features,intervals,timings,split,file_id,dialog,humour,context,timings2,...,audio_path,video_path,test,video_timings,audio_timings,speaker,absolute,text,humour_label,bbox
0,[finally in desperation they went and they con...,"[[finally, in, desperation, they, went, and, t...","[[[0.0, 0.339], [0.339, 0.459], [0.459, 1.157]...","(24.933, 28.206)",test,11318,1017,CONTEXT,True,"[(0.0, 4.37), (5.019, 12.252), (13.519, 14.717...",...,../../data/urfunny2_audio/11318.wav,../../data/urfunny2_videos/11318.mp4,lol,"(24.933, 28.206)","(0.0, 28.206)",None,28.206,finally in desperation they went and they cons...,CONTEXT,"[[[102, 22, 144, 198]], [[102, 22, 141, 199]],..."
1,they gave it back to the wise old woman,"[they, gave, it, back, to, the, wise, old, woman]","[[28.415, 28.475], [28.475, 28.565], [28.565, ...","(28.415, 29.273)",test,11318,1017,1,False,"(28.415, 29.273)",...,../../data/urfunny2_audio/11318.wav,../../data/urfunny2_videos/11318.mp4,lol,"(28.415, 29.273)","(28.415, 29.273)",None,0.858,they gave it back to the wise old woman,Humor,"[[[83, 22, 130, 211]], [[83, 24, 130, 210]], [..."
2,[the most fundamental way in which the third s...,"[[the, most, fundamental, way, in, which, the,...","[[[0.0, 0.06], [0.06, 0.299], [0.319, 1.427], ...","(21.92, 25.841)",test,11319,1017,CONTEXT,True,"[(0.0, 7.273), (8.022, 15.475), (16.233, 21.57...",...,../../data/urfunny2_audio/11319.wav,../../data/urfunny2_videos/11319.mp4,lol,"(21.92, 25.841)","(0.0, 25.841)",None,25.841,the most fundamental way in which the third si...,CONTEXT,"[[[48, 22, 184, 221]], [[46, 20, 185, 223]], [..."
3,and as the saying goes when angry you will mak...,"[and, as, the, saying, goes, when, angry, you,...","[[26.889, 27.039], [27.039, 27.188], [27.188, ...","(26.889, 29.772)",test,11319,1017,1,False,"(26.889, 29.772)",...,../../data/urfunny2_audio/11319.wav,../../data/urfunny2_videos/11319.mp4,lol,"(26.889, 29.772)","(26.889, 29.772)",None,2.883,and as the saying goes when angry you will mak...,Humor,"[[[70, 20, 198, 224]], [[70, 21, 198, 222]], [..."
4,[i tried to remember to go to the balcony],"[[i, tried, to, remember, to, go, to, the, bal...","[[[0.0, 0.18], [0.18, 0.499], [0.499, 0.559], ...","(0.0, 2.365)",test,11321,1017,CONTEXT,True,"[(0.0, 2.365)]",...,../../data/urfunny2_audio/11321.wav,../../data/urfunny2_videos/11321.mp4,lol,"(0.0, 2.365)","(0.0, 2.365)",None,2.365,i tried to remember to go to the balcony,CONTEXT,"[[[66, 20, 168, 222]], [[66, 18, 157, 226]], [..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20327,so i found that they had a friend a local acco...,"[so, i, found, that, they, had, a, friend, a, ...","[[56.142, 56.482], [56.482, 56.781], [56.881, ...","(56.142, 61.69)",val,10945,949,0,False,"(56.142, 61.69)",...,../../data/urfunny2_audio/10945.wav,../../data/urfunny2_videos/10945.mp4,lol,"(56.142, 61.69)","(56.142, 61.69)",None,5.548,so i found that they had a friend a local acco...,Non-Humor,"[[[30, 29, 78, 223], [122, 23, 192, 223], [71,..."
20328,[so their scores had gone up from zero to perc...,"[[so, their, scores, had, gone, up, from, zero...","[[[0.0, 0.359], [0.359, 0.489], [0.489, 0.878]...","(24.644, 30.062)",val,10946,949,CONTEXT,True,"[(0.0, 5.777), (6.146, 6.824), (8.381, 13.928)...",...,../../data/urfunny2_audio/10946.wav,../../data/urfunny2_videos/10946.mp4,lol,"(24.644, 30.062)","(0.0, 30.062)",None,30.062,so their scores had gone up from zero to perce...,CONTEXT,"[[[59, 25, 187, 224]], [[58, 26, 186, 224]], [..."
20329,just say to them cool,"[just, say, to, them, cool]","[[31.389, 31.558], [31.558, 31.708], [31.708, ...","(31.389, 32.177)",val,10946,949,0,False,"(31.389, 32.177)",...,../../data/urfunny2_audio/10946.wav,../../data/urfunny2_videos/10946.mp4,lol,"(31.389, 32.177)","(31.389, 32.177)",None,0.788,just say to them cool,Non-Humor,"[[[0, 6, 164, 220]], [[0, 5, 165, 220]], [[0, ..."
20330,"[the worst in, they used everything that they ...","[[the, worst, in], [the

In [4]:
type(df.iloc[1]['bbox'])

list

In [11]:
weights = torch.sort(
    torch.Tensor(
        list(
            dict(
                sorted(
                    (dict(1 / np.sqrt((df["emotion"].value_counts()))).items())
                )
            ).values()
        )
    )
).values
weights = weights / weights.sum()
weights

tensor([0.1293, 0.1345, 0.1673, 0.1688, 0.1723, 0.2279])

In [16]:
# in weights array swap 0 and 1


tensor([0.1345, 0.1293, 0.1688, 0.1673, 0.1723, 0.2279])

In [13]:
labels = df["emotion"].value_counts()
class_counts = torch.Tensor(
    list(dict(sorted((dict((labels)).items()))).values())
).to(int)
class_counts

tensor([1708, 1849, 1084, 1103, 1041,  595])

In [7]:
dict(sorted((dict(1 / np.sqrt((df["emotion"].value_counts()))).items()))).values()

dict_values([0.024196695924791362, 0.023255813953488372, 0.03037283696153935, 0.03011010318298389, 0.030993786368687184, 0.040996003084539386])

In [7]:
emotions = [
        "neutral",
        "frustration",
        "sadness",
        "anger",
        "excited",
        "happiness",
    ]
wanted_label2id = {emotion: idx for idx, emotion in enumerate(emotions)}
wanted_id2label = {val: key for key, val in wanted_label2id.items()}

In [8]:
wanted_label2id

{'neutral': 0,
 'frustration': 1,
 'sadness': 2,
 'anger': 3,
 'excited': 4,
 'happiness': 5}

In [16]:
wanted_id2label

{0: 'neutral',
 1: 'frustration',
 2: 'sadness',
 3: 'anger',
 4: 'excited',
 5: 'happiness'}

In [9]:
label2id = (
    df.drop_duplicates("emotion_label").set_index("emotion_label").to_dict()["emotion"]
)
id2label = {v: k for k, v in label2id.items()}

In [10]:
label2id

{'frustrated': 0, 'neutral': 1, 'angry': 2, 'happy': 5, 'sad': 3, 'excited': 4}

In [18]:
wanted_id2label  = {0: 'neutral',
 1: 'frustrated',
 2: 'sad',
 3: 'angry',
 4: 'excited',
 5: 'happy'}

In [12]:
curr2want_ids = {
 0: 1,
 1: 0,
 2: 3,
 3: 2,
 4: 4,
 5: 5
 }

In [13]:
df['emotion'].value_counts()

emotion
0    1849
1    1708
2    1103
3    1084
4    1041
5     595
Name: count, dtype: int64

In [14]:
# df['emotion_label'] = df['emotion_label']
df['emotion'] = df['emotion'].apply(lambda x: curr2want_ids[x])
df['emotion'].value_counts()

emotion
1    1849
0    1708
3    1103
2    1084
4    1041
5     595
Name: count, dtype: int64

In [15]:
df['emotion_label'].value_counts()

emotion_label
frustrated    1849
neutral       1708
angry         1103
sad           1084
excited       1041
happy          595
Name: count, dtype: int64

In [20]:
df['emotion_label'] = df['emotion'].apply(lambda x: wanted_id2label[x])
df['emotion_label'].value_counts()

emotion_label
frustrated    1849
neutral       1708
angry         1103
sad           1084
excited       1041
happy          595
Name: count, dtype: int64

In [22]:
df.to_pickle("data/iemo.pkl")

In [24]:
olddf = pd.read_pickle("data/iemo.pkl")
olddf

,talker,sentiment_label,dialog,text_w_one_prev,sentiment,timings,bbox,audio_path,video_path,og_text,len,og_text_with_auth,emotion,emotion_label,index,split,concatenated,text
0,MARY,frustrated,7,MARY: What's he going to say?,0,"(6.21, 9.32)","[[[65, 33, 200, 217]], [[63, 32, 203, 217]], [...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,What's he going to say?,16,MARY: What's he going to say?,1,frustrated,Ses01F_script01_1,train,</s></s> What's he going to say?,</s></s> MARY: What's he going to say?
1,JAMES,neutral,7,MARY: What's he going to say? </s></s> JAMES: ...,1,"(11.9892, 15.1219)","[[[24, 37, 172, 217], [0, 86, 36, 219], [182, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,He saw it.,16,JAMES: He saw it.,0,neutral,Ses01F_script01_1,train,What's he going to say? </s></s> He saw it.,MARY: What's he going to say? </s></s> JAMES: ...
2,JAMES,neutral,7,JAMES: He saw it. </s></s> JAMES: He was out h...,1,"(19.0456, 22.3962)","[[[19, 36, 175, 216], [2, 62, 54, 224], [182, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,He was out here when it broke.,16,JAMES: He was out here when it broke.,0,neutral,Ses01F_script01_1,train,What's he going to say? He saw it. </s></s> ...,MARY: What's he going to say? JAMES: He saw it...
3,JAMES,neutral,7,JAMES: He was out here when it broke. </s></s>...,1,"(24.8, 35.9971)","[[[40, 37, 176, 217], [2, 74, 68, 220], [183, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,Four o clock this morning. I heard a crack a...,16,JAMES: Four o clock this morning. I heard a c...,0,neutral,Ses01F_script01_1,train,What's he going to say? He saw it. He was o...,MARY: What's he going to say? JAMES: He saw it...
4,JAMES,neutral,7,JAMES: Four o clock this morning. I heard a c...,1,"(38.66, 40.53)","[[[27, 35, 168, 218], [184, 129, 222, 205], [0...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,I don't know.,16,JAMES: I don't know.,0,neutral,Ses01F_script01_1,train,What's he going to say? He saw it. He was o...,MARY: What's he going to say? JAMES: He saw it...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7375,WILLIAM,excited,149,"ELIZABETH: Oh, It burnt my comb and all the to...",4,"(215.4, 220.75)","[[[61, 22, 188, 216]], [[60, 20, 187, 215]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,"[LAUGHTER] That was quit a rouser, wasn't it?",16,"WILLIAM: [LAUGHTER] That was quit a rouser, wa...",4,excited,Ses05M_script03_1,test,"I wonder. what, Nobody knows we are here exc...","ELIZABETH: what, Nobody knows we are here exce..."
7376,WILLIAM,excited,149,"WILLIAM: [LAUGHTER] That was quit a rouser, wa...",4,"(221.13, 223.83)","[[[61, 16, 189, 216]], [[61, 15, 188, 216]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,"Oh, I didn't hit you very hard.",16,"WILLIAM: Oh, I didn't hit you very hard.",4,excited,Ses05M_script03_1,test,"I wonder. what, Nobody knows we are here exc...",ELIZABETH: What do we do? WILLIAM: We're all r...
7377,WILLIAM,excited,149,"WILLIAM: Oh, I didn't hit you very hard. </s><...",4,"(229.88, 233.06)","[[[57, 13, 190, 213]], [[56, 16, 190, 215]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,I should never forget his face. [LAUGHTER],16,WILLIAM: I should never forget his face. [LAUG...,4,excited,Ses05M_script03_1,test,"I wonder. what, Nobody knows we are here exc...","WILLIAM: We're all right, aren't we, darling, ..."
7378,WILLIAM,excited,149,WILLIAM: I should never forget his face. [LAUG...,4,"(234.14, 236.66)","[[[62, 11, 190, 215]], [[60, 13, 189, 214]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,We were very much younger then.,16,WILLIAM: We w

In [23]:
df

,talker,sentiment_label,dialog,text_w_one_prev,sentiment,timings,bbox,audio_path,video_path,og_text,len,og_text_with_auth,emotion,emotion_label,index,split,concatenated,text
0,MARY,frustrated,7,MARY: What's he going to say?,0,"(6.21, 9.32)","[[[65, 33, 200, 217]], [[63, 32, 203, 217]], [...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,What's he going to say?,16,MARY: What's he going to say?,1,frustrated,Ses01F_script01_1,train,</s></s> What's he going to say?,</s></s> MARY: What's he going to say?
1,JAMES,neutral,7,MARY: What's he going to say? </s></s> JAMES: ...,1,"(11.9892, 15.1219)","[[[24, 37, 172, 217], [0, 86, 36, 219], [182, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,He saw it.,16,JAMES: He saw it.,0,neutral,Ses01F_script01_1,train,What's he going to say? </s></s> He saw it.,MARY: What's he going to say? </s></s> JAMES: ...
2,JAMES,neutral,7,JAMES: He saw it. </s></s> JAMES: He was out h...,1,"(19.0456, 22.3962)","[[[19, 36, 175, 216], [2, 62, 54, 224], [182, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,He was out here when it broke.,16,JAMES: He was out here when it broke.,0,neutral,Ses01F_script01_1,train,What's he going to say? He saw it. </s></s> ...,MARY: What's he going to say? JAMES: He saw it...
3,JAMES,neutral,7,JAMES: He was out here when it broke. </s></s>...,1,"(24.8, 35.9971)","[[[40, 37, 176, 217], [2, 74, 68, 220], [183, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,Four o clock this morning. I heard a crack a...,16,JAMES: Four o clock this morning. I heard a c...,0,neutral,Ses01F_script01_1,train,What's he going to say? He saw it. He was o...,MARY: What's he going to say? JAMES: He saw it...
4,JAMES,neutral,7,JAMES: Four o clock this morning. I heard a c...,1,"(38.66, 40.53)","[[[27, 35, 168, 218], [184, 129, 222, 205], [0...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,I don't know.,16,JAMES: I don't know.,0,neutral,Ses01F_script01_1,train,What's he going to say? He saw it. He was o...,MARY: What's he going to say? JAMES: He saw it...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7375,WILLIAM,excited,149,"ELIZABETH: Oh, It burnt my comb and all the to...",4,"(215.4, 220.75)","[[[61, 22, 188, 216]], [[60, 20, 187, 215]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,"[LAUGHTER] That was quit a rouser, wasn't it?",16,"WILLIAM: [LAUGHTER] That was quit a rouser, wa...",4,excited,Ses05M_script03_1,test,"I wonder. what, Nobody knows we are here exc...","ELIZABETH: what, Nobody knows we are here exce..."
7376,WILLIAM,excited,149,"WILLIAM: [LAUGHTER] That was quit a rouser, wa...",4,"(221.13, 223.83)","[[[61, 16, 189, 216]], [[61, 15, 188, 216]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,"Oh, I didn't hit you very hard.",16,"WILLIAM: Oh, I didn't hit you very hard.",4,excited,Ses05M_script03_1,test,"I wonder. what, Nobody knows we are here exc...",ELIZABETH: What do we do? WILLIAM: We're all r...
7377,WILLIAM,excited,149,"WILLIAM: Oh, I didn't hit you very hard. </s><...",4,"(229.88, 233.06)","[[[57, 13, 190, 213]], [[56, 16, 190, 215]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,I should never forget his face. [LAUGHTER],16,WILLIAM: I should never forget his face. [LAUG...,4,excited,Ses05M_script03_1,test,"I wonder. what, Nobody knows we are here exc...","WILLIAM: We're all right, aren't we, darling, ..."
7378,WILLIAM,excited,149,WILLIAM: I should never forget his face. [LAUG...,4,"(234.14, 236.66)","[[[62, 11, 190, 215]], [[60, 13, 189, 214]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,We were very much younger then.,16,WILLIAM: We w

In [41]:
def process_intervals(intervals, limit = 4):
    # Initialize the running total and the limit
    # print(intervals , flush = True)
    if  len(intervals)  == 0:
        return (0 , 500)    
    if isinstance(intervals[0][0] , list) or isinstance(intervals[0][0] , np.ndarray):
        
        running_total = 0

        # Initialize the list to store the intervals that fit within the limit
        limited_intervals = []

        # Go through the intervals in reverse order
        for idx_i , i in enumerate(reversed(range(len(intervals)))):
            sublist = intervals[i]
            for j , interval in enumerate(reversed(sublist)):
                # Calculate the absolute difference between end and start
                time_diff = abs(interval[1] - interval[0])
                # If adding this time difference to the running total would make it greater than the limit
                if running_total + time_diff > limit and idx_i == 0 and j == 0:
                    # if on the first iteration the first interval is already greater than the limit
                    limited_intervals.append(interval)
                    break
                elif running_total + time_diff > limit:
                    # Stop the loop
                    break
                
                # Add the time difference to the running total
                running_total += time_diff
                
                # Add the interval to the list
                limited_intervals.append(interval)
            
            # If we haven't reached the limit yet, add the gap to the next sublist (if there is one)
            if i > 0 and running_total < limit:
                gap = intervals[i][0][0] - intervals[i-1][-1][1]
                running_total += gap

        # Reverse the list of intervals to get them in the original order
        limited_intervals = list(reversed(limited_intervals))

        return (limited_intervals[0][0], limited_intervals[-1][1])
    else:
        return (intervals[0][0], intervals[-1][1])

In [42]:
from tqdm import tqdm
tqdm.pandas()

df['timings']  = df['intervals'].progress_apply( lambda x: process_intervals(x , limit = 4))
df['video_timings']  = df['intervals'].progress_apply( lambda x: process_intervals(x , limit = 4))
df['audio_timings']  = df['intervals'].progress_apply( lambda x: process_intervals(x , limit = 29))
df

  0%|          | 0/20332 [00:00<?, ?it/s]

100%|██████████| 20332/20332 [00:00<00:00, 102706.80it/s]


,sentence,features,intervals,timings,split,file_id,root,label,context,timings2,proc_intervals_4,audio_path,video_path,test,video_timings,audio_timings,speaker,absolute
0,[finally in desperation they went and they con...,"[[finally, in, desperation, they, went, and, t...","[[[0.0, 0.339], [0.339, 0.459], [0.459, 1.157]...","(24.933, 28.206)",test,11318,1017,CONTEXT,True,"[(0.0, 4.37), (5.019, 12.252), (13.519, 14.717...","(21.022, 28.206)",../../data/urfunny2_audio/11318.wav,../../data/urfunny2_videos/11318.mp4,lol,"(24.933, 28.206)","(0.0, 28.206)",None,28.206
1,they gave it back to the wise old woman,"[they, gave, it, back, to, the, wise, old, woman]","[[28.415, 28.475], [28.475, 28.565], [28.565, ...","(28.415, 29.273)",test,11318,1017,1,False,"(28.415, 29.273)","(28.415, 29.273)",../../data/urfunny2_audio/11318.wav,../../data/urfunny2_videos/11318.mp4,lol,"(28.415, 29.273)","(28.415, 29.273)",None,0.858
2,[the most fundamental way in which the third s...,"[[the, most, fundamental, way, in, which, the,...","[[[0.0, 0.06], [0.06, 0.299], [0.319, 1.427], ...","(21.92, 25.841)",test,11319,1017,CONTEXT,True,"[(0.0, 7.273), (8.022, 15.475), (16.233, 21.57...","(21.92, 25.841)",../../data/urfunny2_audio/11319.wav,../../data/urfunny2_videos/11319.mp4,lol,"(21.92, 25.841)","(0.0, 25.841)",None,25.841
3,and as the saying goes when angry you will mak...,"[and, as, the, saying, goes, when, angry, you,...","[[26.889, 27.039], [27.039, 27.188], [27.188, ...","(26.889, 29.772)",test,11319,1017,1,False,"(26.889, 29.772)","(26.889, 29.772)",../../data/urfunny2_audio/11319.wav,../../data/urfunny2_videos/11319.mp4,lol,"(26.889, 29.772)","(26.889, 29.772)",None,2.883
4,[i tried to remember to go to the balcony],"[[i, tried, to, remember, to, go, to, the, bal...","[[[0.0, 0.18], [0.18, 0.499], [0.499, 0.559], ...","(0.0, 2.365)",test,11321,1017,CONTEXT,True,"[(0.0, 2.365)]","(0.0, 2.365)",../../data/urfunny2_audio/11321.wav,../../data/urfunny2_videos/11321.mp4,lol,"(0.0, 2.365)","(0.0, 2.365)",None,2.365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20327,so i found that they had a friend a local acco...,"[so, i, found, that, they, had, a, friend, a, ...","[[56.142, 56.482], [56.482, 56.781], [56.881, ...","(56.142, 61.69)",val,10945,949,0,False,"(56.142, 61.69)","(56.142, 61.69)",../../data/urfunny2_audio/10945.wav,../../data/urfunny2_videos/10945.mp4,lol,"(56.142, 61.69)","(56.142, 61.69)",None,5.548
20328,[so their scores had gone up from zero to perc...,"[[so, their, scores, had, gone, up, from, zero...","[[[0.0, 0.359], [0.359, 0.489], [0.489, 0.878]...","(24.644, 30.062)",val,10946,949,CONTEXT,True,"[(0.0, 5.777), (6.146, 6.824), (8.381, 13.928)...","(24.644, 30.062)",../../data/urfunny2_audio/10946.wav,../../data/urfunny2_videos/10946.mp4,lol,"(24.644, 30.062)","(0.0, 30.062)",None,30.062
20329,just say to them cool,"[just, say, to, them, cool]","[[31.389, 31.558], [31.558, 31.708], [31.708, ...","(31.389, 32.177)",val,10946,949,0,False,"(31.389, 32.177)","(31.389, 32.177)",../../data/urfunny2_audio/10946.wav,../../data/urfunny2_videos/10946.mp4,lol,"(31.389, 32.177)","(31.389, 32.177)",None,0.788
20330,"[the worst in, they used everything that they ...","[[the, worst, in], [they, used, everything, th...","[[[0.0, 0.06], [0.06, 0.2], [0.409, 0.559]], [...","(33.354, 37.245)",val,10948,949,CONTEXT,True,"[(0.0, 0.559), (1.896, 5.927), (7.982, 11.963)...","(11.903, 37.245)",../../data/urfunny2_audio/10948.wav,../../data/urfunny2_videos/10948.mp4,lol,"(33.354, 37.245)","(4.929, 37.245)",None,32.316


In [43]:
df.to_pickle("data/urfunny.pkl")

True

In [44]:
df['absolute'] = df['audio_timings'].apply(lambda x: x[1] - x[0])


In [5]:
jdx = 6
df['intervals'].iloc[jdx] , df['audio_timings'].iloc[jdx] , df['video_timings'].iloc[jdx] , df['absolute'] .iloc[jdx]

(array([[[0.   , 0.11 ],
         [0.11 , 0.399],
         [0.399, 0.729],
         [0.729, 0.968],
         [1.068, 1.417]],
 
        [[1.627, 1.686],
         [1.686, 1.876],
         [1.876, 2.165],
         [2.165, 2.345],
         [2.345, 3.003]]]),
 (array([0.  , 0.11]), array([1.686, 1.876])),
 (array([0.  , 0.11]), array([1.686, 1.876])),
 array([1.686, 1.766]))

In [9]:
df[ len(df['absolute'])  == 1 ]

KeyError: False

In [24]:
df[df['absolute'].apply(lambda x: not isinstance(x, int))]

,sentence,features,intervals,timings,split,file_id,root,label,context,timings2,proc_intervals_4,audio_path,video_path,test,video_timings,audio_timings,speaker,absolute
0,[finally in desperation they went and they con...,"[[finally, in, desperation, they, went, and, t...","[[[0.0, 0.339], [0.339, 0.459], [0.459, 1.157]...","(24.933, 28.206)",test,11318,1017,CONTEXT,True,"[(0.0, 4.37), (5.019, 12.252), (13.519, 14.717...","(21.022, 28.206)",../../data/urfunny2_audio/11318.wav,../../data/urfunny2_videos/11318.mp4,lol,"(24.933, 28.206)","(0.0, 28.206)",None,28.206
1,they gave it back to the wise old woman,"[they, gave, it, back, to, the, wise, old, woman]","[[28.415, 28.475], [28.475, 28.565], [28.565, ...","(28.415, 29.273)",test,11318,1017,1,False,"(28.415, 29.273)","(28.415, 29.273)",../../data/urfunny2_audio/11318.wav,../../data/urfunny2_videos/11318.mp4,lol,"(28.415, 29.273)","(28.415, 29.273)",None,0.858
2,[the most fundamental way in which the third s...,"[[the, most, fundamental, way, in, which, the,...","[[[0.0, 0.06], [0.06, 0.299], [0.319, 1.427], ...","(21.92, 25.841)",test,11319,1017,CONTEXT,True,"[(0.0, 7.273), (8.022, 15.475), (16.233, 21.57...","(21.92, 25.841)",../../data/urfunny2_audio/11319.wav,../../data/urfunny2_videos/11319.mp4,lol,"(21.92, 25.841)","(0.0, 25.841)",None,25.841
3,and as the saying goes when angry you will mak...,"[and, as, the, saying, goes, when, angry, you,...","[[26.889, 27.039], [27.039, 27.188], [27.188, ...","(26.889, 29.772)",test,11319,1017,1,False,"(26.889, 29.772)","(26.889, 29.772)",../../data/urfunny2_audio/11319.wav,../../data/urfunny2_videos/11319.mp4,lol,"(26.889, 29.772)","(26.889, 29.772)",None,2.883
4,[i tried to remember to go to the balcony],"[[i, tried, to, remember, to, go, to, the, bal...","[[[0.0, 0.18], [0.18, 0.499], [0.499, 0.559], ...","(0.0, 2.365)",test,11321,1017,CONTEXT,True,"[(0.0, 2.365)]","(0.0, 2.365)",../../data/urfunny2_audio/11321.wav,../../data/urfunny2_videos/11321.mp4,lol,"(0.0, 2.365)","(0.0, 2.365)",None,2.365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20327,so i found that they had a friend a local acco...,"[so, i, found, that, they, had, a, friend, a, ...","[[56.142, 56.482], [56.482, 56.781], [56.881, ...","(56.142, 61.69)",val,10945,949,0,False,"(56.142, 61.69)","(56.142, 61.69)",../../data/urfunny2_audio/10945.wav,../../data/urfunny2_videos/10945.mp4,lol,"(56.142, 61.69)","(56.142, 61.69)",None,5.548
20328,[so their scores had gone up from zero to perc...,"[[so, their, scores, had, gone, up, from, zero...","[[[0.0, 0.359], [0.359, 0.489], [0.489, 0.878]...","(24.644, 30.062)",val,10946,949,CONTEXT,True,"[(0.0, 5.777), (6.146, 6.824), (8.381, 13.928)...","(24.644, 30.062)",../../data/urfunny2_audio/10946.wav,../../data/urfunny2_videos/10946.mp4,lol,"(24.644, 30.062)","(0.0, 30.062)",None,30.062
20329,just say to them cool,"[just, say, to, them, cool]","[[31.389, 31.558], [31.558, 31.708], [31.708, ...","(31.389, 32.177)",val,10946,949,0,False,"(31.389, 32.177)","(31.389, 32.177)",../../data/urfunny2_audio/10946.wav,../../data/urfunny2_videos/10946.mp4,lol,"(31.389, 32.177)","(31.389, 32.177)",None,0.788
20330,"[the worst in, they used everything that they ...","[[the, worst, in], [they, used, everything, th...","[[[0.0, 0.06], [0.06, 0.2], [0.409, 0.559]], [...","(33.354, 37.245)",val,10948,949,CONTEXT,True,"[(0.0, 0.559), (1.896, 5.927), (7.982, 11.963)...","(11.903, 37.245)",../../data/urfunny2_audio/10948.wav,../../data/urfunny2_videos/10948.mp4,lol,"(33.354, 37.245)","(4.929, 37.245)",None,32.316


(53.548, 56.821)

In [ ]:
isinstance(a[0][0] , list)

In [ ]:
process_intervals(df['intervals'].iloc[1])